In [2]:
from pathlib import Path
from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from rates.isotope import Isotope
from rates import Kadonis, Reaclib

from ppnmontecarlo import NetworkSetup

kadonis1 = Kadonis(version=1)
kadonis03 = Kadonis(version=0.3)

reaclib_ppn = Reaclib(version="PPN_2")

excel_path = "Rates.xlsx"

In [3]:
isos = ['AL 27-factor',
 'AR 38-factor',
 'AR 39-factor',
 'AR 40-factor',
 'AR 41-factor',
 'AR 42-factor',
 'CA 48-factor',
 'CL 37-factor',
 'CL 38-factor',
 'CL 39-factor',
 'MG 26-factor',
 'MG 27-factor',
 'NA 23-factor',
 'NE 20-factor',
 'NE 22-factor',
 'P  31-factor',
 'S  35-factor',
 'S  36-factor',
 'SI 28-factor',
 'SI 30-factor',
 'SI 31-factor',
 'SI 32-factor']

In [6]:
ns = NetworkSetup(Path("networksetup.txt"))

with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    workbook  = writer.book

    for iso in isos:
        iso = iso[0:5]
        index = ns.reaction_index(["NEUT ", iso, ], "(n,g)")
        source = ns.reaction_network[index].source
        
        print(iso, source)
        
        if source == "KADON":
            
            iso = iso.replace(' ', '')
            url = f"https://exp-astro.de/kadonis1.0/selementquery.php?isotope={iso}"
            tables = pd.read_html(url)
            
            tables[4] = tables[4].sort_values(2, ascending=False)
            
            link = ["link",]

            for i in list(tables[4][5])[1:]:
                link.append(f"https://exp-astro.de/kadonis1.0/refquery.php?reference={i}")

            tables[4][6] = link
            
            # Write Tables
            
            tables[3] = pd.DataFrame(np.vstack([tables[3].columns, tables[3]]))
            
            tables[3].to_excel(writer, sheet_name=iso, startrow=1, startcol=0, header=False, index=False)
            tables[4].to_excel(writer, sheet_name=iso, startrow=1, startcol=6, header=False, index=False)
            
            worksheet = writer.sheets[iso]
            worksheet.write_url('A1', url, string=iso)
            
            # Format Columns
            
            worksheet.set_column('A:A', 8)
            worksheet.set_column('B:D', 11)
            
            worksheet.set_column('G:H', 11)
            worksheet.set_column('I:J', 5)
            worksheet.set_column('K:K', 52)
            worksheet.set_column('L:M', 15)
            
            # Format Rows
            
            title_row_format = workbook.add_format({'bold': True, 'bg_color': "#5B9BD5", 'align': "center", 'valign': "vcenter", 'text_wrap': True})
            odd_row_format = workbook.add_format({'bg_color': "#BDD7EE"})
            
            columns = list("ABCDGHIJKLM")
            rows = range(3, 50)
            
            worksheet.set_row(0, 34, title_row_format)
            worksheet.set_row(1, 34, title_row_format)
            
            for r in rows:
                if r % 2 != 0:
                    worksheet.set_row(r, 15, odd_row_format)
                    
            for i in range(len(tables[4])+2, 50):
                worksheet.set_row(i, 15, odd_row_format)
            
            worksheet.set_column('E:F', None, odd_row_format)
            worksheet.set_column('M:Z', None, odd_row_format)
        
        else:
            print(source)
        
        


AL 27 KADON
AR 38 KADON
AR 39 KADON
AR 40 KADON
AR 41 JINAC
JINAC
AR 42 JINAC
JINAC
CA 48 KADON
CL 37 KADON
CL 38 JINAC
JINAC
CL 39 JINAC
JINAC
MG 26 KADON
MG 27 JINAC
JINAC
NA 23 KADON
NE 20 KADON
NE 22 KADON
P  31 KADON
S  35 JINAC
JINAC
S  36 KADON
SI 28 KADON
SI 30 KADON
SI 31 JINAC
JINAC
SI 32 JINAC
JINAC


In [28]:
ns = NetworkSetup(Path("networksetup.txt"))
uncert_df = pd.read_csv("uncertainties.csv")

isotopes = []
isotopes2 = []
lines = []

for iso in isos:
    iso = iso[0:5]
    
    if source == "JINAC":
        print(iso)
        element = iso[0:2].capitalize().strip()
        mass = int(iso[2:].strip())
        
        isotopes.append(f"{mass}{element}")
        
        isotopes2.append(f"{element.lower()}{mass}")
        
        print(element, mass)
        #print(uncert_df[(uncert_df.Element == element) & (uncert_df.Mass == mass)])
        
        print(reaclib_ppn.get_n_gamma(iso).label)
        
        print(reaclib_ppn.get_n_gamma(iso))

print(isotopes2)
reaclib_ppn.df[(reaclib_ppn.df.E0 == "n") & (reaclib_ppn.df.E1.isin(isotopes2)) & (reaclib_ppn.df.E3 == "") & (reaclib_ppn.df.SetLabel != "ka02")]

AL 27
Al 27
ka02
n+Al27 -> Al28
AR 38
Ar 38
ka02
n+Ar38 -> Ar39
AR 39
Ar 39
ka02
n+Ar39 -> Ar40
AR 40
Ar 40
ka02
n+Ar40 -> Ar41
AR 41
Ar 41
rath
n+Ar41 -> Ar42
AR 42
Ar 42
rath
n+Ar42 -> Ar43
CA 48
Ca 48
ka02
n+Ca48 -> Ca49
CL 37
Cl 37
ka02
n+Cl37 -> Cl38
CL 38
Cl 38
rath
n+Cl38 -> Cl39
CL 39
Cl 39
rath
n+Cl39 -> Cl40
MG 26
Mg 26
ka02
n+Mg26 -> Mg27
MG 27
Mg 27
rath
n+Mg27 -> Mg28
NA 23
Na 23
ka02
n+Na23 -> Na24
NE 20
Ne 20
ka02
n+Ne20 -> Ne21
NE 22
Ne 22
ka02
n+Ne22 -> Ne23
P  31
P 31
ka02
n+P31 -> P32
S  35
S 35
rath
n+S35 -> S36
S  36
S 36
ka02
n+S36 -> S37
SI 28
Si 28
ka02
n+Si28 -> Si29
SI 30
Si 30
ka02
n+Si30 -> Si31
SI 31
Si 31
rath
n+Si31 -> Si32
SI 32
Si 32
rath
n+Si32 -> Si33
['al27', 'ar38', 'ar39', 'ar40', 'ar41', 'ar42', 'ca48', 'cl37', 'cl38', 'cl39', 'mg26', 'mg27', 'na23', 'ne20', 'ne22', 'p31', 's35', 's36', 'si28', 'si30', 'si31', 'si32']


,Chapter,E0,E1,E2,E3,E4,E5,SetLabel,RateType,ReverseRate,QValue,Rate,Reaction
29123,4,n,mg27,mg28,,,,rath,,,8.504,"[4.73336, 0.0227938, -4.06138, 12.6524, -1.018...",n+Mg27 -> Mg28
29340,4,n,si31,si32,,,,rath,,,9.203,"[11.9451, -0.0144696, 0.154735, 0.304696, 0.02...",n+Si31 -> Si32
29343,4,n,si32,si33,,,,rath,,,4.483,"[12.845, -0.0507754, 3.87479, -6.3252, 0.49132...",n+Si32 -> Si33
29554,4,n,s35,s36,,,,rath,,,9.889,"[17.7117, -0.0335945, 2.69164, -8.06087, 0.981...",n+S35 -> S36
29675,4,n,cl38,cl39,,,,rath,,,8.073,"[8.16108, 0.00308986, -2.10317, 8.96391, -1.03...",n+Cl38 -> Cl39
29678,4,n,cl39,cl40,,,,rath,,,5.829,"[12.8602, -0.0270037, 1.38817, -1.52329, 0.097...",n+Cl39 -> Cl40
29798,4,n,ar41,ar42,,,,rath,,,9.426,"[20.0136, -0.0496343, 4.39661, -11.9273, 1.059...",n+Ar41 -> Ar42
29801,4,n,ar42,ar43,,,,rath,,,5.627,"[7.98561, 0.021426, -3.02977, 8.06247, -0.4144...",n+Ar42 -> Ar43


In [11]:
uncert_df = pd.read_csv("uncertainties.csv")

uncert_df[uncert_df.Isotope.isin(isotopes)]

,Isotope,Element,Mass,MaxLDensity,MaxGStrength,MaxRate,Charge,Neutron,MaxFactor,MinLDensity,MinGStrength,MinRate,MinFactor,ExperimentalUncert(%)
28,20Ne,Ne,20,-1,-1,4.750000e+03,10,10,NaN,-1,-1,4750.00,NaN,13.342697
30,22Ne,Ne,22,-1,-1,4.000000e+02,10,12,NaN,-1,-1,400.00,NaN,5.076142
38,23Na,Na,23,-1,-1,7.400000e+03,11,12,NaN,-1,-1,7400.00,NaN,2.781955
46,26Mg,Mg,26,-1,-1,1.480000e+03,12,14,NaN,-1,-1,1480.00,NaN,7.149758
47,27Mg,Mg,27,2,4,3.255520e+05,12,15,1.020128,6,1,81299.90,0.254756,NaN
53,27Al,Al,27,-1,-1,8.260000e+04,13,14,NaN,-1,-1,82600.00,NaN,16.926230
60,28Si,Si,28,-1,-1,1.920000e+04,14,14,NaN,-1,-1,19200.00,NaN,9.186603
62,30Si,Si,30,-1,-1,4.860000e+04,14,16,NaN,-1,-1,48600.00,NaN,18.134328
63,31Si,Si,31,6,3,3.725920e+05,14,17,1.027209,4,1,110001.00,0.303265,NaN
64,32Si,Si,32,0,0,5.412550e+04,14,18,1.000000,1,1,8064.68,0.149000,NaN


In [45]:
reaclib_ppn.get_n_gamma("AR 41").label

'rath'